In [2]:
# !pip install langchain
# !pip install langchain-openai
# !pip install langchain-core

In [3]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# Or if you are in Colab, uncoment below and add your api key
# import os
# os.environ["OPENAI_API_KEY"] = "your-api-key"

# Common Sense Agent

1. Model (large language model like ChatGPT, or Claude)
2. Prompt (system message and the user input)
3. Tool calling

In [4]:
from langchain_openai import ChatOpenAI
import os
# os.environ["OPENAI_API_KEY"] = "your openai api key"
llm = ChatOpenAI(model="gpt-4o", temperature=0)

llm.invoke("Hi")

AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'stop', 'logprobs': None}, id='run-4dd77c54-5eb6-4255-b3ef-3e16d65856bf-0')

In [5]:
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Answer this question: {question} in bullet points. Output:\n")

In [6]:
chain = prompt | llm

chain

ChatPromptTemplate(input_variables=['question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='Answer this question: {question} in bullet points. Output:\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10ed00dd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10d9add90>, model_name='gpt-4o', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy='')

In [7]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [8]:
output = chain.invoke("Write down names of cats.")
output

AIMessage(content='- Whiskers\n- Mittens\n- Shadow\n- Luna\n- Simba\n- Oliver\n- Bella\n- Charlie\n- Max\n- Daisy', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 23, 'total_tokens': 55}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'stop', 'logprobs': None}, id='run-928ceb6d-03b2-4305-9dd6-9a376de558de-0')

In [9]:
from IPython.display import Markdown

Markdown(output.content)

- Whiskers
- Mittens
- Shadow
- Luna
- Simba
- Oliver
- Bella
- Charlie
- Max
- Daisy

In [12]:
from langchain_core.tools import tool
from serpapi import GoogleSearch

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}

query = "Cool buildings in Paris"
search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
search.get_dict()["organic_results"]

[{'position': 1,
  'title': '25 Must-See Paris Landmarks',
  'link': 'https://www.architecturaldigest.com/gallery/paris-architectural-landmarks',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.architecturaldigest.com/gallery/paris-architectural-landmarks&ved=2ahUKEwjHr-KZr7CHAxUkD1kFHc9ODNwQFnoECBoQAQ',
  'displayed_link': 'https://www.architecturaldigest.com › ... › travel',
  'thumbnail': 'https://serpapi.com/searches/6698eccae4a23dcc9b271a5b/images/b82971cf6568ad27e2fa13ba6a1ef50fcedef7e58fe760b01529225406d3a2a2.jpeg',
  'favicon': 'https://serpapi.com/searches/6698eccae4a23dcc9b271a5b/images/b82971cf6568ad27e2fa13ba6a1ef50f8f513dcefb7186a53f3be813729d0233.png',
  'date': 'Feb 18, 2016',
  'snippet': 'Located on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...',
  'snippet_highlighted_words': ['Paris', 'famous'],
  'images': ['https://serpapi.com

In [13]:
results = search.get_dict()["organic_results"]
contexts = "\n---\n".join(
       ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
   )
contexts

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [14]:
@tool
def web_search(query: str) -> str:
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    results = search.get_dict()["organic_results"]
    contexts = "\n---\n".join(
         ["\n".join([x["title"], x["snippet"], x["link"]]) for x in results]
        )
    
    return contexts

web_search.invoke("Cool buildings in Paris")

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [15]:
llm_with_tool = llm.bind_tools([web_search])

research_chain = prompt | llm_with_tool 

research_chain.invoke("What are the cool buildings in Paris?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AUIXIDFkQvKVzIbccS23BnX3', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 91, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f872ce55-7e51-4e55-8bf6-fa0c678f1fad-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_AUIXIDFkQvKVzIbccS23BnX3'}])

In [16]:
sample_search_result = research_chain.invoke("What are the cool buildings in Paris?")
sample_search_result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ygXHT4c9H9LUPe9hBpo9NpyW', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 91, 'total_tokens': 108}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c3f01e05-4e7d-4afb-af26-ae8a68b8292b-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_ygXHT4c9H9LUPe9hBpo9NpyW'}])

In [17]:
sample_search_result.tool_calls

[{'name': 'web_search',
  'args': {'query': 'cool buildings in Paris'},
  'id': 'call_ygXHT4c9H9LUPe9hBpo9NpyW'}]

In [18]:
sample_search_result.tool_calls[0]['args']

{'query': 'cool buildings in Paris'}

In [19]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args'])

research_chain.invoke("What are the cool buildings in Paris?")

{'query': 'cool buildings in Paris'}

In [20]:
research_chain = prompt | llm_with_tool | (lambda x: x.tool_calls[0]['args']) | web_search

research_chain.invoke("What are the cool buildings in Paris?")

"25 Must-See Paris Landmarks\nLocated on the Île de la Cité, a thin island in the Seine, Notre Dame de Paris is perhaps the most famous cathedral in the world. Characterized ...\nhttps://www.architecturaldigest.com/gallery/paris-architectural-landmarks\n---\nTHE 10 BEST Paris Architectural Buildings (Updated 2024)\nArchitectural Buildings in Paris · 1. Louvre Museum · 2. Cathédrale Notre-Dame de Paris · 3. Arc de Triomphe · 4. Palais Garnier · 5. Sainte- ...\nhttps://www.tripadvisor.com/Attractions-g187147-Activities-c47-t3-Paris_Ile_de_France.html\n---\nParis City Guide: 23 Places Every Architect Must Visit\n1. Centre Georges Pompidou ... Save this picture! ... Description: This is one of the most iconic buildings in Paris and houses the Musée National d ...\nhttps://www.archdaily.com/922278/23-places-in-paris-every-architect-must-visit\n---\n22 of the Most Famous Buildings in Paris You Must See\nMost Famous Buildings in Paris You Must See · 1. The Louvre Museum · 2. Notre-Dame Cathed

In [21]:
def fake_agent(query: str) -> str:
    search_result = research_chain.invoke("What are the cool buildings in Paris?")
    fake_agent = llm.invoke(f"The user query was: {query}. The web search result for\
        that is: {search_result}. Give a summarized answer:")
    
    return fake_agent.content

fake_agent("What are the cool buildings in Paris?")

'Paris is home to numerous iconic and architecturally significant buildings. Here are some must-see landmarks:\n\n1. **Louvre Museum** - Famous for its art collection and the glass pyramid entrance.\n2. **Notre-Dame Cathedral** - Renowned Gothic cathedral on Île de la Cité.\n3. **Arc de Triomphe** - Monument honoring those who fought and died for France.\n4. **Palais Garnier** - Opulent opera house.\n5. **Sainte-Chapelle** - Known for its stunning stained glass windows.\n6. **Centre Georges Pompidou** - Modern art museum with a unique architectural design.\n7. **Basilica of the Sacré-Cœur** - White-domed basilica located at the highest point in the city.\n8. **Fondation Louis Vuitton** - Contemporary art museum with striking modern architecture.\n9. **Panthéon** - Mausoleum containing the remains of distinguished French citizens.\n10. **Villa Savoye** - Iconic modernist villa designed by Le Corbusier.\n\nThese buildings showcase a mix of historical, modern, and contemporary architectur

In [22]:
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", "You are a helpful assistant"),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
    ]
                                          )

llm = ChatOpenAI(model="gpt-4o", temperature=0)

tools = [web_search]

llm_with_tools = llm.bind_tools(tools)

research_agent = (
                 RunnablePassthrough.assign(
                    agent_scratchpad=lambda x: format_to_tool_messages(x['intermediate_steps'])
                    )
) | prompt | llm_with_tools | ToolsAgentOutputParser()

research_agent.invoke({"input": "What are the cool buildings in Paris?", "intermediate_steps": []})

[ToolAgentAction(tool='web_search', tool_input={'query': 'cool buildings in Paris'}, log="\nInvoking: `web_search` with `{'query': 'cool buildings in Paris'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mhFPgbbfJ8p31MYYkcoHJOBg', 'function': {'arguments': '{"query":"cool buildings in Paris"}', 'name': 'web_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 87, 'total_tokens': 104}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_c4e5b6fa31', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3688d97e-592c-4fbb-90cc-44d756a63f02-0', tool_calls=[{'name': 'web_search', 'args': {'query': 'cool buildings in Paris'}, 'id': 'call_mhFPgbbfJ8p31MYYkcoHJOBg'}])], tool_call_id='call_mhFPgbbfJ8p31MYYkcoHJOBg')]

In [23]:
prompt

ChatPromptTemplate(input_variables=['input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': [], 'agent_scratchpad': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input']

In [24]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=research_agent,tools=tools)
agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': 'Alicante, Spain, is home to several cool and notable buildings. Here are some of the highlights:\n\n1. **Castle of Santa Bárbara**: A historic castle offering panoramic views of the city and the Mediterranean Sea.\n2. **Casa Carbonell**: A remarkable architectural gem and historic building.\n3. **The Town Hall (Ayuntamiento de Alicante)**: An impressive baroque building located in the city center.\n4. **The Central Market (Mercado Central)**: A bustling market housed in a beautiful early 20th-century building.\n5. **Teatro Principal**: The main theater in Alicante, known for its neoclassical architecture.\n6. **Bullring (Plaza de Toros)**: A historic bullring that also hosts various events.\n7. **Diputacion Provincial de Alicante**: Another notable building with significant architectural value.\n8. **Archives Municipales**: The municipal archives housed in a historic building.\n9. **Sarrio Tower**: A unique tower with 

Wwe can automate part of this process by using a built in method in LangChain
that abstracts away the chaining procedure.

In [25]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub

llm = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [web_search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)
agent_executor.invoke({'input': 'What are cool buildings in Alicante Spain?'})

{'input': 'What are cool buildings in Alicante Spain?',
 'output': "Alicante, Spain, is home to several cool and notable buildings. Here are some of the highlights:\n\n1. **Castle of Santa Bárbara**: A historic castle offering panoramic views of the city and the Mediterranean Sea.\n2. **Casa Carbonell**: A remarkable architectural gem and historic building.\n3. **Diputacion Provincial de Alicante**: The provincial government building with impressive architecture.\n4. **Archives Municipales**: The municipal archives housed in a notable building.\n5. **Sarrio Tower**: A distinctive tower in the city.\n6. **Oficina De Correos**: The main post office building with unique architectural features.\n7. **The Town Hall**: An important civic building with historical significance.\n8. **The Central Market**: A bustling market housed in a beautiful building.\n9. **Teatro Principal**: The main theater in Alicante, known for its architectural beauty.\n10. **Bullring (Plaza de Toros)**: A historic bu

In [26]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold